<a href="https://colab.research.google.com/github/AhmedSSoliman/Get-relevant-Arabic-Paragraph-for-a-question/blob/master/Get_the_most_relevant_paragraph_for_the_proposed_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get the most relevant paragraph for the proposed question in Arabic

In [ ]:
%%capture
!pip install pandas nltk pandas sklearn

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources for tokenization (if not already downloaded)
nltk.download('punkt')

# Load your CSV file into a DataFrame
csv_file_path = '/content/similarity.csv'
df = pd.read_csv(csv_file_path)

df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Paragraph,Question1,Question2,Question3,Question4,Question5
0,صناعة الورق على مستوى العالم، ده موضوع كبير وم...,التصنيع ده بيستخدم ايه بشكل كتير اوي؟,ايه المجال اللي بتخش فيه الصناعة؟,ليه بيحاولوا يطورو التصينع ده؟,ايه المشاكل اللي بتقف قدام الصناعة دية؟,ايه المشاكل اللي بتيجي بسبب ان البني ادمين بق...
1,الأشجار، دي حاجة أساسية في حياتنا وبيئتنا. الأ...,ازاي الشجر بيساعد في صحة الاشخاص؟,ايه علاقة الشجر بالمية؟,الشجر بيأثر علي التلوثات؟,ايه المشاكل اللي موجودة قدام اني ازرع شجر؟,ايه علاقة المشروعات بالشجر؟
2,الولادة، ده موضوع مهم وحيوي في حياة كل أسرة مص...,لما حد بيولد بتبقي ايه؟,ايه هي الاشكال اللي ممكن يولدوا بيها الستات؟,ليه الست ترضع طبيعي احسن؟,ليه يبقي احسن ان الست تعمل تحاليل قبل ما تولد؟,ليه لازم يكون في عملية تحسين الوضع المزاجي لل...
3,المكياج، ده موضوع بيهم كتير من الناس في مصر. ا...,الميكاب عبارة عن ايه؟,انواع الميكاب ايه؟,تقدر تقولي زمان كانوا عندهم ميكاب ولا لا؟,هو لازم يشيلوا الميكب قبل ما يناموا؟,الميكب ليه اكتر من نوع براند؟
4,دور المرأة في المجتمع المصري ده موضوع مهم وعمي...,ايه هو عمل البنات في المجتمعات؟,ايه علاقة البنات بالشغل؟,هو البنات بتمثل؟,البنات بتزرع؟,البنات بتخترع حاجات؟
5,الثعابين على مستوى العالم، ده موضوع شيق ومهم. ...,ايه هي التعابين؟,هو في انواع تعابين فيها سم وانواع لا؟,عايز اعرف حجم التعابين,التعابين بتسيطر علي ايه؟,ايه علاقة التعابين بالحضارة؟
6,الحياة في بريطانيا في الستينيات، ده كان وقت مل...,ايه هي اسامى الباندات اللي كان ليها بصمة علي ا...,الصناعة كانت كويسة فى الفترة دية؟,علاقة البلاد كانت متطورة وقتها؟,اخبار اللبس ايه في الفترة دية؟,احوال العلام ايه في الوقت دة؟
7,السجائر، ده موضوع مهم وليه تأثير كبير على الصح...,ايه هي محتوايات السجارة؟,ايه اللي موجود في السجارة يخلي الواحد مدمن؟,ايه المرض اللي بيجي من السجارة؟,ايه علاقة الزيادة في الضريبة و ان واحد يبطل شرب؟,شرب السجارة مؤثر علي الشغل؟
8,القهوة، دي حاجة مهمة جداً في حياتنا وخصوصاً في...,لو انا مقريف اشرب ايه؟,هو المشروبات اللي فيها كافيين بيساعد علي الكت...,ايه اشكال المشروبات اللي فيها كافيين المنتشرة؟,لما يجيلي ضيوف ينفع اشربهم مشروبات فيها كافيين؟,هي الاشخاص بتشرب ايه الصبح؟
9,الخضار، دي حاجة أساسية في الأكل المصري ومهمة ل...,اكل الخضراوات بيدينا ايه؟,الخضراوات بيمنع عننا مرض زي ايه؟,الخضراوات بتتاكل في الصيام؟,في مطعم بيركزعلي انه يقدم الخضراوات؟,الخضراوات بيخش في العلاج الطبيعي؟


In [ ]:
# Get the column names
columns = df.columns
all_questions = []


# Iterate over each row
for index, row in df.iterrows():
   # Iterate over each column (i.e., each question)
   for column in columns[1:]:
       # Print the question
       all_questions.append(row[column])
print(all_questions)

['التصنيع ده بيستخدم ايه بشكل كتير اوي؟', ' ايه المجال اللي بتخش فيه الصناعة؟', ' ليه بيحاولوا يطورو التصينع ده؟', ' ايه المشاكل اللي بتقف قدام الصناعة دية؟', ' ايه المشاكل اللي بتيجي بسبب ان البني ادمين بقت تستعمل الجهاز الرقمي بدل الاوراق؟', 'ازاي الشجر بيساعد في صحة الاشخاص؟', ' ايه علاقة الشجر بالمية؟', ' الشجر بيأثر علي التلوثات؟', ' ايه المشاكل اللي موجودة قدام اني ازرع شجر؟', ' ايه علاقة المشروعات بالشجر؟', 'لما حد بيولد بتبقي ايه؟', ' ايه هي الاشكال اللي ممكن يولدوا بيها الستات؟', ' ليه الست ترضع طبيعي احسن؟', ' ليه يبقي احسن ان الست تعمل تحاليل قبل ما تولد؟', ' ليه لازم يكون في عملية تحسين الوضع المزاجي للست قبل ما تولد؟', 'الميكاب عبارة عن ايه؟', ' انواع الميكاب ايه؟', ' تقدر تقولي زمان كانوا عندهم ميكاب ولا لا؟', ' هو لازم يشيلوا الميكب قبل ما يناموا؟', ' الميكب ليه اكتر من نوع براند؟', 'ايه هو عمل البنات في المجتمعات؟', ' ايه علاقة البنات بالشغل؟', ' هو البنات بتمثل؟', ' البنات بتزرع؟', ' البنات بتخترع حاجات؟', 'ايه هي التعابين؟', ' هو في انواع تعابين فيها سم وانواع لا؟', '

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# Load the model
model = SentenceTransformer('all-MiniLM-L6-v2')

csv_file = "/content/similarity.csv"
# Read the CSV file
data = pd.read_csv(csv_file)

# Extract the paragraphs and questions from the CSV file

# Define the PDF document as a list of paragraphs
paragraphs = data['Paragraph'].tolist()
# Define the questions as a list
questions = data[['Question1', 'Question2', 'Question3', 'Question4', 'Question5']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).tolist()


# Sentence Transformers and Mean Pooling

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
#sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')


question = " هو في انواع تعابين فيها سم وانواع لا؟"
# Tokenize sentences
encoded_input = tokenizer(paragraphs, padding=True, truncation=True, return_tensors='pt')
encoded_question = tokenizer(question, padding=True, truncation=True, return_tensors='pt')
# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    model_output_question = model(**encoded_question)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
sentence_embeddings_question = mean_pooling(model_output_question, encoded_question['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
sentence_embeddings_question = F.normalize(sentence_embeddings_question, p=2, dim=1)
print("Sentence embeddings:")
print(sentence_embeddings)
print("Sentence embeddings question:")
print(sentence_embeddings_question)


tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

Sentence embeddings:
tensor([[ 0.0626, -0.0092, -0.0056,  ...,  0.0895, -0.0565, -0.0116],
        [ 0.0742,  0.0089,  0.0065,  ...,  0.0823, -0.0588, -0.0052],
        [ 0.0707, -0.0011,  0.0142,  ...,  0.0555, -0.0688, -0.0191],
        ...,
        [ 0.0443,  0.0366,  0.0215,  ...,  0.0832, -0.0357,  0.0087],
        [ 0.0579, -0.0105, -0.0094,  ...,  0.0905, -0.0686, -0.0217],
        [ 0.0648,  0.0003, -0.0031,  ...,  0.0773, -0.0571, -0.0062]])
Sentence embeddings question:
tensor([[-2.5687e-02,  7.0536e-02, -4.0711e-03, -8.2563e-02,  4.5347e-02,
         -5.1774e-02,  6.1929e-02, -6.7851e-02,  3.3615e-02,  6.9421e-02,
          2.0412e-02, -2.6468e-02,  2.2574e-02,  1.0532e-02,  7.0856e-02,
         -3.1717e-02, -2.3655e-02, -5.5810e-02, -9.6155e-02, -6.0710e-02,
          1.6947e-02,  3.5804e-02,  7.2307e-02, -4.4752e-02,  2.1086e-02,
         -4.4704e-02,  3.3878e-03,  4.5904e-02, -6.1163e-02,  2.6324e-02,
          2.3365e-02, -1.4164e-02,  4.3647e-02, -2.8673e-03,  1.4649e-0

In [ ]:

# Compute similarity scores of the question to every paragraph
cos_scores = util.pytorch_cos_sim(sentence_embeddings_question, sentence_embeddings)[0]

# Find the paragraph with the highest cosine similarity score
top_results = torch.topk(cos_scores, k=1)

# Print the question and the most relevant paragraph
print("Question:", question)
print("\nMost relevant paragraph:", paragraphs[top_results.indices[0]], "(Score: %.4f)" % top_results.values[0])

Question:  هو في انواع تعابين فيها سم وانواع لا؟

Most relevant paragraph: الحياة في بريطانيا في الستينيات، ده كان وقت مليان بالتغييرات والتطورات الكبيرة. في الستينيات، بريطانيا شهدت تغييرات اجتماعية وثقافية هايلة.
أولاً، الستينيات في بريطانيا كانت فترة الانفتاح والتحرر الثقافي. كان فيه تحول كبير في القيم والعادات، والناس بدأت تتقبل أفكار جديدة وأساليب حياة مختلفة.
وكمان، الفن والموسيقى كان ليهم دور كبير في الستينيات. فرق زي البيتلز والرولينج ستونز ظهرت وأثرت على الموسيقى العالمية. الستينيات كانت فترة ذهبية للموسيقى البريطانية.
وفي الستينيات، بريطانيا شهدت تطورات اقتصادية وصناعية. كان فيه نمو في الصناعات وتحسينات في الاقتصاد، وده ساعد في تحسين مستوى الحياة للناس.
وكمان، الستينيات في بريطانيا كانت فترة التغييرات السياسية. كان فيه تحول في السياسات وظهور حركات اجتماعية جديدة كان ليها تأثير كبير على المجتمع.
وفي الستينيات في بريطانيا، كانت فيه ثورة في مجال الأزياء والموضة. الأزياء بقت أكثر جرأة وتحرر، وظهرت تصميمات جديدة وأنيقة كان ليها تأثير كبير على الموضة العالمية.
وكمان، السينما والتلي

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

class ParagraphMatcher:
    def __init__(self, model_name):
       self.tokenizer = AutoTokenizer.from_pretrained(model_name)
       self.model = AutoModel.from_pretrained(model_name)

    def encode(self, texts):
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
        with torch.no_grad():
            outputs = self.model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

    def calculate_similarities(self, question_embedding, paragraph_embeddings):
        similarities = torch.matmul(question_embedding, paragraph_embeddings.T)
        return similarities

    def get_most_relevant_paragraph(self, paragraphs, question):
        if not paragraphs or all(not p.strip() for p in paragraphs):
            raise ValueError("Paragraphs cannot be empty or contain only whitespace.")

        # Compute embeddings for the question and paragraphs
        paragraph_embeddings = self.encode(paragraphs)
        question_embedding = self.encode(question)

        # Calculate the similarity scores between the question embedding and each paragraph embedding
        similarities = self.calculate_similarities(question_embedding, paragraph_embeddings)

        # Identify the paragraph with the highest similarity score as the most relevant paragraph
        most_relevant_paragraph_index = torch.argmax(similarities)
        most_relevant_paragraph = paragraphs[most_relevant_paragraph_index]
        score = similarities[0][most_relevant_paragraph_index].item()

        return most_relevant_paragraph, score

    def test_paragraph_matcher(self, paragraphs, question):
       most_relevant_paragraph, score = self.get_most_relevant_paragraph(paragraphs, question)
       print(f"Most relevant paragraph: {most_relevant_paragraph}")
       print(f"Score: {score} ")


In [ ]:
# Create a ParagraphMatcher instance
#matcher = ParagraphMatcher("sentence-transformers/bert-base-nli-mean-tokens")
matcher = ParagraphMatcher("bert-base-uncased")
# Define some sample paragraphs and a question
#paragraphs = ["This is the first paragraph.", "This is the second paragraph.", "This is the third paragraph."]
question = " هو في انواع تعابين فيها سم وانواع لا؟"
#question = "What were the main challenges you faced, and how did you address them given the limited time?"
# Test the ParagraphMatcher
matcher.test_paragraph_matcher(paragraphs, question)

Most relevant paragraph: يا جماعة، النهاردة هنتكلم عن موضوع مهم وقريب لقلوبنا كلنا، وهو الأطفال. الأطفال في مصر دول مصدر الفرحة والحيوية في كل بيت. من أول ما يفتحوا عيونهم للدنيا، بيبقى ليهم تأثير سحري على الكبار. الطفل المصري بيتربى في جو عائلي دافئ، حيث العيلة بتكون متماسكة والكل بيشارك في تربيته.
أول حاجة بنلاحظها في الأطفال هي البراءة والفضول. يعني الطفل دايماً عنده فضول يعرف كل حاجة حواليه. بيسأل أسئلة كتير، ودايماً بيحب يكتشف العالم اللي حواليه. والحلو في الأمر ده إنه بيخلي الكبار يشوفوا الدنيا بعيون جديدة، عيون مليانة استغراب ودهشة.
وكمان الأطفال في مصر بيحبوا اللعب والضحك. يعني تلاقي الأطفال دايماً في الشوارع والحدائق بيلعبوا مع بعض، والضحكة مش بتفارق وشوشهم. اللعب ده مش بس وسيلة تسلية، ده كمان طريقة ليهم عشان يتعلموا ويطوروا مهاراتهم. والكورة بالذات هي اللعبة المفضلة لكتير من الأطفال في مصر، وبتعتبر جزء من الثقافة المصرية.
لكن مش كل حاجة بتكون سهلة في تربية الأطفال. يعني في تحديات كتير بتواجه الأهالي والمعلمين. أول حاجة هي التعليم. التعليم في مصر بيواجه صعوبات، زي الفصول المزد

# BERTScore

In [ ]:
!pip install transformers
!pip install bert-score


In [ ]:
from transformers import AutoTokenizer
from bert_score import BERTScorer

class ParagraphMatcher:
   def __init__(self, model_name):
       self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)
       self.scorer = BERTScorer(model_type=model_name)

   def get_most_relevant_paragraph(self, paragraphs, question):
       if not paragraphs or all(not p.strip() for p in paragraphs):
           raise ValueError("Paragraphs cannot be empty or contain only whitespace.")

       # Initialize the maximum score and the most relevant paragraph
       max_score = -1
       most_relevant_paragraph = None

       # Iterate over each paragraph
       for paragraph in paragraphs:
           # Calculate the BERTScore between the question and the paragraph
           P, R, F1 = self.scorer.score([question], [paragraph])
           score = F1.mean().item()

           # Update the maximum score and the most relevant paragraph if necessary
           if score > max_score:
               max_score = score
               most_relevant_paragraph = paragraph

       return most_relevant_paragraph, max_score

   def test_paragraph_matcher(self, paragraphs, question):
       most_relevant_paragraph, score = self.get_most_relevant_paragraph(paragraphs, question)
       print(f"Most relevant paragraph: {most_relevant_paragraph}")
       print(f"Score: {score} ")


In [ ]:
import pandas as pd
csv_file = "/content/similarity.csv"
# Read the CSV file
data = pd.read_csv(csv_file)

# Extract the paragraphs and questions from the CSV file

# Define the PDF document as a list of paragraphs
paragraphs = data['Paragraph'].tolist()
# Define the questions as a list
questions = data[['Question1', 'Question2', 'Question3', 'Question4', 'Question5']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).tolist()


In [ ]:
# Create a ParagraphMatcher instance
#matcher = ParagraphMatcher("sentence-transformers/bert-base-nli-mean-tokens")
matcher = ParagraphMatcher("bert-base-uncased")
# Define some sample paragraphs and a question
#paragraphs = ["This is the first paragraph.", "This is the second paragraph.", "This is the third paragraph."]
question = " هو في انواع تعابين فيها سم وانواع لا؟"
#question = "What were the main challenges you faced, and how did you address them given the limited time?"
# Test the ParagraphMatcher
matcher.test_paragraph_matcher(paragraphs, question)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Most relevant paragraph: القهوة، دي حاجة مهمة جداً في حياتنا وخصوصاً في مصر. القهوة مش بس مشروب، ده جزء من الثقافة وليها تاريخ كبير ومؤثر.
أولاً، القهوة دي مشروب محبوب جداً في مصر. يعني، في كل حتة هتلاقي قهاوي وناس بتشرب قهوة في الصبح وطول اليوم. القهوة بتكون ليها دور كبير في التجمعات الاجتماعية والنقاشات.
وكمان، فيه أنواع كتير من القهوة. يعني، في القهوة العربي، القهوة التركي، وأنواع تانية زي الاسبريسو والكابتشينو. كل نوع ليه طريقة تحضير مختلفة وطعم مميز.
وفي مصر، القهوة بتكون جزء من العادات اليومية. يعني، كتير من الناس بيبدأوا يومهم بفنجان قهوة، وده بيكون ليه تأثير على مزاجهم ونشاطهم.
وكمان، القهوة ليها تأثير على الصحة. يعني، فيه دراسات بتقول إن القهوة ممكن تكون مفيدة للصحة بشروط معينة، زي تقليل خطر بعض الأمراض.
وفي العالم، القهوة بتكون ليها دور اقتصادي كبير. يعني، زراعة القهوة وتجارتها بتكون مصدر دخل لكتير من الدول والمزارعين.
وفي مصر، القهوة بتعتبر جزء أساسي من الضيافة. يعني، لما تروح بيت حد، أول حاجة بيعرضوا عليك هي فنجان القهوة. ده بيعبر عن الكرم والترحيب.
وكمان، فيه طقوس معينة لت

# Cosine Similarity

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import csv

class ParagraphMatcher:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def get_most_relevant_paragraph(self, paragraphs, question):
        if not paragraphs or all(not p.strip() for p in paragraphs):
            raise ValueError("Paragraphs cannot be empty or contain only whitespace.")

        # Encode the question and paragraphs
        question_embedding = self.model.encode([question])
        paragraph_embeddings = self.model.encode(paragraphs)

        # Calculate the cosine similarity between the question and each paragraph
        similarities = cosine_similarity(question_embedding, paragraph_embeddings)

        # Find the index of the most relevant paragraph
        most_relevant_index = similarities.argmax()

        return paragraphs[most_relevant_index], similarities[0][most_relevant_index]

    def test_paragraph_matcher(self, paragraphs, question):
        most_relevant_paragraph, score = self.get_most_relevant_paragraph(paragraphs, question)
        return {
            "most_relevant_paragraph": most_relevant_paragraph,
            "score": score
        }




    def test_paragraph_matcher_AllQuestions(self, paragraphs, questions, output_file):
        with open(output_file, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Question', 'Most Relevant Paragraph'])  # Write the header
                for question in question_set:
                    most_relevant_paragraph, score = self.get_most_relevant_paragraph(paragraphs, question)
                    writer.writerow([question, most_relevant_paragraph])  # Write the question and the most relevant paragraph to the CSV

In [ ]:
# Create a ParagraphMatcher instance
matcher = ParagraphMatcher("sentence-transformers/bert-base-nli-mean-tokens")
#matcher = ParagraphMatcher("bert-base-uncased")
# Define some sample paragraphs and a question
question = " هو في انواع تعابين فيها سم وانواع لا؟"
# Test the ParagraphMatcher
matcher.test_paragraph_matcher(paragraphs, question)

{'most_relevant_paragraph': 'الثعابين على مستوى العالم، ده موضوع شيق ومهم. الثعابين دي مجموعة متنوعة من الزواحف وليها تأثير كبير على البيئة والثقافات حوالين العالم. في مصر، الثعابين موجودة وليها دور في البيئة الطبيعية.\nأولاً، الثعابين بتتنوع في الأحجام والأشكال. من الثعابين الصغيرة للكبيرة زي الأناكوندا والبوا، كل نوع ليه خصائصه وطريقة عيشه.\nوفي البيئات المختلفة، الثعابين بتقوم بأدوار مهمة. بتساعد في السيطرة على أعداد الفئران والحشرات، وبالتالي بتكون ليها دور في الحفاظ على التوازن البيئي.\nوكمان، فيه ثعابين سامة وغير سامة. الثعابين السامة زي الكوبرا والراتلسنيك بتكون مهمة في دراسات السموم وتطوير الأمصال.\nوفي الثقافة، الثعابين ليها دور رمزي كبير. في كتير من الحضارات، الثعبان بيتم تصويره كرمز للحكمة أو الشر، وبيظهر في الأساطير والفنون.\nفي مصر، الثعبان ليه مكانة خاصة في التاريخ والثقافة. يعني، الثعبان كان رمز للملكية في مصر القديمة وبيظهر في كتير من النقوش والتماثيل.\nوفي مصر، بنلاقي أنواع مختلفة من الثعابين في مناطق متفرقة. يعني، في الصحرا وعلى السواحل وفي الأراضي الزراعية، كل منطقة 

In [ ]:
# Create a ParagraphMatcher instance
matcher = ParagraphMatcher("sentence-transformers/bert-base-nli-mean-tokens")
#matcher = ParagraphMatcher("bert-base-uncased")
# Define some sample paragraphs and a question
question = " علاقة البلاد كانت متطورة وقتها؟"
#question = "What were the main challenges you faced, and how did you address them given the limited time?"
# Test the ParagraphMatcher
matcher.test_paragraph_matcher(paragraphs, question)

{'most_relevant_paragraph': 'الثعابين على مستوى العالم، ده موضوع شيق ومهم. الثعابين دي مجموعة متنوعة من الزواحف وليها تأثير كبير على البيئة والثقافات حوالين العالم. في مصر، الثعابين موجودة وليها دور في البيئة الطبيعية.\nأولاً، الثعابين بتتنوع في الأحجام والأشكال. من الثعابين الصغيرة للكبيرة زي الأناكوندا والبوا، كل نوع ليه خصائصه وطريقة عيشه.\nوفي البيئات المختلفة، الثعابين بتقوم بأدوار مهمة. بتساعد في السيطرة على أعداد الفئران والحشرات، وبالتالي بتكون ليها دور في الحفاظ على التوازن البيئي.\nوكمان، فيه ثعابين سامة وغير سامة. الثعابين السامة زي الكوبرا والراتلسنيك بتكون مهمة في دراسات السموم وتطوير الأمصال.\nوفي الثقافة، الثعابين ليها دور رمزي كبير. في كتير من الحضارات، الثعبان بيتم تصويره كرمز للحكمة أو الشر، وبيظهر في الأساطير والفنون.\nفي مصر، الثعبان ليه مكانة خاصة في التاريخ والثقافة. يعني، الثعبان كان رمز للملكية في مصر القديمة وبيظهر في كتير من النقوش والتماثيل.\nوفي مصر، بنلاقي أنواع مختلفة من الثعابين في مناطق متفرقة. يعني، في الصحرا وعلى السواحل وفي الأراضي الزراعية، كل منطقة 

In [ ]:
output_file = 'results.csv'
matcher.test_paragraph_matcher_AllQuestions(paragraphs, all_questions, output_file)